In [1]:
# Import the necessary packages
import pandas as pd
#import os
#import sqlalchemy
from sqlalchemy import create_engine
import pymysql
# Import the raw_data
raw_data=pd.read_csv("C:\Data_projects\Cotisation\cotisantsircantec_typecoll_nbagents.csv",sep=";")


In [2]:
#pip install pymysql

In [3]:
# Data exploration

# Quick look at the dataFrame
raw_data.head(10)
# Show the numerber of lines and rows
raw_data.shape
# Check the type of each column
raw_data["annee"].dtype  #int64
raw_data["famille_d_employeurs"].dtype #object
raw_data["effectif_cotisants"].dtype #float64
# Check the uniques values of each column
raw_data["annee"].unique()
raw_data["effectif_cotisants"].unique()
raw_data["famille_d_employeurs"].unique()
#print the overview of the file
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   annee                 40 non-null     int64  
 1   famille_d_employeurs  40 non-null     object 
 2   effectif_cotisants    40 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


TO DO : Data cleaning
1-convert effectif_cotisants into int
2-convert famille_d_employeurs into str
3-update '4 Hospitalière' to 'hospitalier'
4-update '3 Territoriale (salariés)' to 'contractuel'
5-update '2 Territoriale (élus)' to 'elu'
6-update '1 Etat' to 'etat'
7-update '5 Autres employeurs' to 'autre'
8-create a new column "montant cotisation" which is:300 0000 euro  for hospitalier, 90 000 euro for contractuel, 140 000 euro for elu, 54 450 000 euro  for etat, 789 000 euro for autre.
9-create a new column "matricule" 
1= etat, 2 =elu, 3 =contarctuel, 4= hospitalier, 5= autre
10-save the csv file as cleaned_data at "C:\Data_projects\Cotisation"

In [4]:
#Convert effectif_cotisants into int
raw_data["effectif_cotisants"]=raw_data["effectif_cotisants"].astype('int')

#Verify that effectif_cotisants is now an integer
assert raw_data["effectif_cotisants"].dtype == 'int'

#Convert famille_d_employeurs into str
raw_data["famille_d_employeurs"]=raw_data["famille_d_employeurs"].astype('string')

#Verify that famille_d_employeurs is now a string
assert raw_data["famille_d_employeurs"].dtype == 'string'


# Update '4 Hospitalière' to 'hospitalier'
raw_data["famille_d_employeurs"]=raw_data["famille_d_employeurs"].str.replace('4 Hospitalière','hospitalier')
# Update '3 Territoriale (salariés)' to 'contractuel'
raw_data["famille_d_employeurs"]=raw_data["famille_d_employeurs"].str.replace('3 Territoriale (salariés)','contractuel')
# Update '2 Territoriale (élus)' to 'elu'
raw_data["famille_d_employeurs"]=raw_data["famille_d_employeurs"].str.replace('2 Territoriale (élus)','elu')
# update '1 Etat' to 'etat'
raw_data["famille_d_employeurs"]=raw_data["famille_d_employeurs"].str.replace('1 Etat','etat')
# Update '5 Autres employeurs' to 'autre'
raw_data["famille_d_employeurs"]=raw_data["famille_d_employeurs"].str.replace('5 Autres employeurs','autre')

# create a new column "montant cotisation" create a new copy of famille_d_employeurs then replace the values and the type of the column.
raw_data["montant_cotisation"]=raw_data["famille_d_employeurs"]
raw_data["montant_cotisation"]=raw_data["montant_cotisation"].str.replace('hospitalier','300000')
raw_data["montant_cotisation"]=raw_data["montant_cotisation"].str.replace('contractuel','90000')
raw_data["montant_cotisation"]=raw_data["montant_cotisation"].str.replace('elu','140000')
raw_data["montant_cotisation"]=raw_data["montant_cotisation"].str.replace('etat','54450000')
raw_data["montant_cotisation"]=raw_data["montant_cotisation"].str.replace('autre','789000')
raw_data["montant_cotisation"]=raw_data["montant_cotisation"].astype('float')


# same for matricule
raw_data["matricule"]=raw_data["famille_d_employeurs"]
raw_data["matricule"]=raw_data["matricule"].str.replace('etat','3256')
raw_data["matricule"]=raw_data["matricule"].str.replace('elu','4578')
raw_data["matricule"]=raw_data["matricule"].str.replace('contractuel','7514')
raw_data["matricule"]=raw_data["matricule"].str.replace('hospitalier','1021')
raw_data["matricule"]=raw_data["matricule"].str.replace('autre','45687')
raw_data["matricule"]=raw_data["matricule"].astype('int')

In [5]:
# Save the csv file as cleaned_data.sql in 
#define connection properties
user="root"
password="root"
host="localhost"
db_name="ircantec"
port=3306
db_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}',pool_recycle=port)

# Persist the data
raw_data.to_sql(
name="cleaned_data",
con=db_engine,
if_exists="replace",
index=False
#index_label="id"
)

40

# TO DO : Data Transformation
3-update and add 325 to "montant cotisation" of year  "2014"
4-update and add 700 to "montant cotisation" of year  "2020"

In [6]:
#Verify the persistance
stock_data = pd.read_sql("SELECT * FROM ircantec.cleaned_data", db_engine)
stock_data

,annee,famille_d_employeurs,effectif_cotisants,montant_cotisation,matricule
0,2015,hospitalier,547586,300000.0,1021
1,2016,contractuel,1140683,90000.0,7514
2,2017,hospitalier,569744,300000.0,1021
3,2018,elu,202117,140000.0,4578
4,2018,hospitalier,582109,300000.0,1021
5,2018,autre,257900,789000.0,45687
6,2019,etat,859546,54450000.0,3256
7,2014,etat,799960,54450000.0,3256
8,2014,contractuel,1164955,90000.0,7514
9,2015,etat,789838,54450000.0,3256
